# 약물 재목적화 시스템 평가

이 노트북은 약물 재목적화 시스템의 기본 성능을 평가합니다.

## 평가 방법
- Leave-one-out 평가: 알려진 약물-질병 관계를 하나씩 숨기고 시스템이 이를 복구할 수 있는지 테스트
- Hit@K 메트릭: 상위 K개 후보 중에 실제 약물이 포함되는 비율


In [ ]:
import sys
import os
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import matplotlib.pyplot as plt
import seaborn as sns

# 프로젝트 루트를 Python 경로에 추가
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

from src.service import RepurposeService
from src.data_loader import DataLoader


## 1. 데이터 로드 및 서비스 초기화


In [ ]:
# 서비스 초기화
service = RepurposeService("data")
service.initialize()

print("서비스 초기화 완료")
print(f"약물 수: {len(service.get_all_drugs())}")
print(f"질병 수: {len(service.get_all_diseases())}")

# 알려진 약물-질병 관계 확인
data_loader = service.data_loader
known_relations = data_loader.drug_disease_df
print(f"\n알려진 약물-질병 관계 수: {len(known_relations)}")
print("\n알려진 관계들:")
for _, row in known_relations.iterrows():
    drug = data_loader.get_drug_by_id(row['drug_id'])
    disease = data_loader.get_disease_by_id(row['disease_id'])
    print(f"- {drug['drug_name']} → {disease['disease_name']} ({row['evidence']})")


## 2. Leave-one-out 평가 함수


In [ ]:
def leave_one_out_evaluation(service: RepurposeService, k_values: List[int] = [1, 3, 5, 10]) -> Dict:
    """
    Leave-one-out 평가를 수행합니다.
    
    Args:
        service: RepurposeService 인스턴스
        k_values: 평가할 K 값들
        
    Returns:
        평가 결과 딕셔너리
    """
    data_loader = service.data_loader
    known_relations = data_loader.drug_disease_df
    
    results = {}
    
    for k in k_values:
        hits = 0
        total_tests = 0
        test_details = []
        
        print(f"\n=== K={k} 평가 중 ===")
        
        for idx, row in known_relations.iterrows():
            drug_id = row['drug_id']
            disease_id = row['disease_id']
            
            # 해당 관계를 임시로 제거
            original_df = data_loader.drug_disease_df.copy()
            data_loader.drug_disease_df = data_loader.drug_disease_df.drop(idx)
            
            # 그래프 재구축
            service.graph_builder.build_graph()
            
            # 질병 이름으로 랭킹 수행
            disease = data_loader.get_disease_by_id(disease_id)
            disease_name = disease['disease_name']
            
            try:
                rankings = service.rank_for_disease(disease_name, top_k=k)
                
                # 상위 K개에 해당 약물이 있는지 확인
                ranked_drug_ids = [r['drug_id'] for r in rankings]
                hit = drug_id in ranked_drug_ids
                
                if hit:
                    hits += 1
                    rank = ranked_drug_ids.index(drug_id) + 1
                else:
                    rank = None
                
                test_details.append({
                    'drug_id': drug_id,
                    'disease_id': disease_id,
                    'drug_name': data_loader.get_drug_by_id(drug_id)['drug_name'],
                    'disease_name': disease_name,
                    'hit': hit,
                    'rank': rank,
                    'total_candidates': len(rankings)
                })
                
                total_tests += 1
                
                print(f"  {data_loader.get_drug_by_id(drug_id)['drug_name']} → {disease_name}: {'HIT' if hit else 'MISS'} (rank: {rank})")
                
            except Exception as e:
                print(f"  오류 발생: {drug_id} → {disease_id}: {e}")
                
            finally:
                # 원본 데이터 복원
                data_loader.drug_disease_df = original_df
        
        hit_rate = hits / total_tests if total_tests > 0 else 0
        results[k] = {
            'hit_rate': hit_rate,
            'hits': hits,
            'total_tests': total_tests,
            'test_details': test_details
        }
        
        print(f"Hit@{k}: {hit_rate:.3f} ({hits}/{total_tests})")
    
    return results


## 3. 평가 실행


In [ ]:
# Leave-one-out 평가 실행
k_values = [1, 3, 5, 10]
evaluation_results = leave_one_out_evaluation(service, k_values)

print("\n=== 최종 평가 결과 ===")
for k in k_values:
    result = evaluation_results[k]
    print(f"Hit@{k}: {result['hit_rate']:.3f} ({result['hits']}/{result['total_tests']})")
